In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Importing database
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

### Training a neural network using leaky ReLU

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [5]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6426 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8202
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5553 - accuracy: 0.8158 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5339 - accuracy: 0.8225 - val_loss: 0.5157 - val_accuracy:

### Implementing ELU in TensorFlow

In [6]:
keras.layers.Dense(10, activation="elu")

### Implementing SELU

During training, a neural network composed exclusively of a stack of dense layers using the SELU activation function and LeCun initialization will self-normalize: the output of each layer will tend to preserve the same mean and variance during training, which solves the vanishing/exploding gradients problem. As a result, this activation function outperforms the other activation functions very significantly for such neural nets, so you should really try it out. Unfortunately, the self-normalizing property of the SELU activation function is easily broken: you cannot use ℓ1 or ℓ2 regularization, regular dropout, max-norm, skip connections or other non-sequential topologies (so recurrent neural networks won't self-normalize). However, in practice it works quite well with sequential CNNs. If you break self-normalization, SELU will not necessarily outperform other activation functions.

By default, the SELU hyperparameters (scale and alpha) are tuned in such a way that the mean output of each neuron remains close to 0, and the standard deviation remains close to 1 (assuming the inputs are standardized with mean 0 and standard deviation 1 too). Using this activation function, even a 1,000 layer deep neural network preserves roughly mean 0 and standard deviation 1 across all layers, avoiding the exploding/vanishing gradients problem:

In [7]:
# Using a neural net for Fashion MNIST with 100 hidden layers, using the SELU activation function:

np.random.seed(42)
tf.random.set_seed(42)

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

Now let's train it. Do not forget to scale the inputs to mean 0 and standard deviation 1:



In [3]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [11]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 61s 34ms/step - loss: 1.3466 - accuracy: 0.4814 - val_loss: 1.7223 - val_accuracy: 0.3852
Epoch 2/5
1719/1719 [==============================] - 60s 35ms/step - loss: 1.2287 - accuracy: 0.5148 - val_loss: 0.8012 - val_accuracy: 0.7128
Epoch 3/5
1719/1719 [==============================] - 60s 35ms/step - loss: 0.7622 - accuracy: 0.7122 - val_loss: 0.7106 - val_accuracy: 0.7222
Epoch 4/5
1719/1719 [==============================] - 60s 35ms/step - loss: 0.6481 - accuracy: 0.7567 - val_loss: 0.6208 - val_accuracy: 0.7712
Epoch 5/5
1719/1719 [==============================] - 58s 34ms/step - loss: 0.6011 - accuracy: 0.7767 - val_loss: 0.5882 - val_accuracy: 0.7938


If we use ReLU with the same architecture it will suffer from the vanishing/exploding gradients problem.

## Batch Normalization

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_105 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_106 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [14]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [15]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8750 - accuracy: 0.7122 - val_loss: 0.5523 - val_accuracy: 0.8222
Epoch 2/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5753 - accuracy: 0.8030 - val_loss: 0.4724 - val_accuracy: 0.8470
Epoch 3/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5189 - accuracy: 0.8204 - val_loss: 0.4374 - val_accuracy: 0.8546
Epoch 4/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4827 - accuracy: 0.8325 - val_loss: 0.4151 - val_accuracy: 0.8602
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4565 - accuracy: 0.8408 - val_loss: 0.3996 - val_accuracy: 0.8642
Epoch 6/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4397 - accuracy: 0.8471 - val_loss: 0.3867 - val_accuracy: 0.8692
Epoch 7/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4242 - accuracy: 0.8513 - val_loss: 0.3764 - val_ac

## Transfer Learning

Let's split the fashion MNIST training set in two:

X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).
X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.
The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused

In [16]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [17]:
tf.random.set_seed(42)
np.random.seed(42)

In [18]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [19]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [20]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5927 - accuracy: 0.8101 - val_loss: 0.3897 - val_accuracy: 0.8662
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3523 - accuracy: 0.8784 - val_loss: 0.3289 - val_accuracy: 0.8819
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3170 - accuracy: 0.8896 - val_loss: 0.3014 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2972 - accuracy: 0.8973 - val_loss: 0.2892 - val_accuracy: 0.9018
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2834 - accuracy: 0.9021 - val_loss: 0.2774 - val_accuracy: 0.9058
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2728 - accuracy: 0.9062 - val_loss: 0.2731 - val_accuracy: 0.9073
Epoch 7/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2641 - accuracy: 0.9092 - val_loss: 0.2717 - val_accuracy:

In [21]:
model_A.save("my_model_A.h5")

In [22]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

Note that model_B_on_A and model_A actually share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build model_B_on_A on top of a clone of model_A:

In [23]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [24]:
# Al ppio se freeza el entrenamiento de las capas reutilizadas durante algunas epocas para darle tiempo a las capas
# sin entrenar de aprender pesos representativos
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [25]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 35ms/step - loss: 2.2058 - accuracy: 0.1200 - val_loss: 2.0775 - val_accuracy: 0.1927
Epoch 2/4
7/7 [==============================] - 0s 20ms/step - loss: 2.0266 - accuracy: 0.1600 - val_loss: 1.9250 - val_accuracy: 0.2241
Epoch 3/4
7/7 [==============================] - 0s 21ms/step - loss: 1.8670 - accuracy: 0.1950 - val_loss: 1.7802 - val_accuracy: 0.2546
Epoch 4/4
7/7 [==============================] - 0s 21ms/step - loss: 1.7161 - accuracy: 0.2300 - val_loss: 1.6428 - val_accuracy: 0.2890
Epoch 1/16
7/7 [==============================] - 0s 33ms/step - loss: 1.2177 - accuracy: 0.3400 - val_loss: 0.7654 - val_accuracy: 0.4807
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.6158 - accuracy: 0.6050 - val_loss: 0.5251 - val_accuracy: 0.7069
Epoch 3/16
7/7 [==============================] - 0s 20ms/step - loss: 0.4191 - accuracy: 0.8000 - val_loss: 0.3891 - val_accuracy: 0.8489
Epoch 4/16
7/7 [===============

In [26]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0986 - accuracy: 0.9815


[0.0985935777425766, 0.9815000295639038]

Con el modelo B entrenado de 0, el accuracy seria 0.97, mientras que aca es 0.98 (en el libro da 0.99 - dice que probo la mejor estructura). Tener en cuenta que transfer learning no funciona bien para redes densas pequeñas.
Transfer learning works best with deep convolutional neural networks!

# Faster Optimizers

## Momentum optimization

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

## Nesterov Accelerated Gradient

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

## AdaGrad

In [ ]:
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

## RMSProp

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

## Adam Optimization

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Adamax Optimization

In [ ]:
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Nadam Optimization

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Learning Rate Scheduling

Implementing power scheduling in keras is the easiest option: just set the decay hyperparameter when creating an optimizer.
The decay is the inverse of s (the number of steps it takes to divide the learning rate by one more unit), and Keras assumes that c is equal to 1

In [4]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [6]:
n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/25
1719/1719 [==============================] - 13s 6ms/step - loss: 0.4855 - accuracy: 0.8282 - val_loss: 0.4183 - val_accuracy: 0.8586
Epoch 2/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3777 - accuracy: 0.8664 - val_loss: 0.3782 - val_accuracy: 0.8734
Epoch 3/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3449 - accuracy: 0.8773 - val_loss: 0.3690 - val_accuracy: 0.8714
Epoch 4/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3239 - accuracy: 0.8849 - val_loss: 0.3625 - val_accuracy: 0.8798
Epoch 5/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3081 - accuracy: 0.8897 - val_loss: 0.3513 - val_accuracy: 0.8760
Epoch 6/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2947 - accuracy: 0.8947 - val_loss: 0.3529 - val_accuracy: 0.8742
Epoch 7/25
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2850 - accuracy: 0.8974 - val_loss: 0.3411 - val_accuracy

In [7]:
# ver mas opciones de scheduling en el libro o en el notebook

## Avoiding Overfitting Through Regularization

### l1 and l2 regularization 

In [8]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor of 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

### Dropout

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 15s 9ms/step - loss: 0.5661 - accuracy: 0.8032 - val_loss: 0.3769 - val_accuracy: 0.8596
Epoch 2/2
1719/1719 [==============================] - 13s 8ms/step - loss: 0.4234 - accuracy: 0.8450 - val_loss: 0.3520 - val_accuracy: 0.8688


### Max norm

In [ ]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))